In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pymongo
from splinter import Browser
import pandas as pd
from selenium import webdriver

In [2]:
# NASA Mars News
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [3]:
html = requests.get(url)

In [4]:
soup = BeautifulSoup(html.text, 'html.parser')

In [5]:
news_title = soup.find("div", class_="content_title").findChildren()[0].string.strip()
news_p = soup.find("div", class_="rollover_description_inner").text.strip()

In [6]:
# JPL Mars Space Images - Featured Image

In [7]:
browser = webdriver.Chrome()

In [8]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.get(url)
time.sleep(3)

In [9]:
browser.find_element_by_link_text('FULL IMAGE').click()

In [10]:
time.sleep(2)
browser.find_element_by_link_text('more info').click()
time.sleep(5)

In [11]:
soup = BeautifulSoup(browser.page_source, "html.parser")

In [12]:
featured_image_url = "https://www.jpl.nasa.gov" + soup.find("img", class_="main_image")['src']
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17563_hires.jpg'

In [13]:
# Mars Weather

In [43]:
url = "https://twitter.com/marswxreport?lang=en"
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')

In [55]:
mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mars_weather = mars_weather.split(',')
mars_weather

['Sol 2054 (May 17',
 ' 2018)',
 ' Sunny',
 ' high 4C/39F',
 ' low -72C/-97F',
 ' pressure at 7.40 hPa',
 ' daylight 05:21-17:20']

In [16]:
url = "https://space-facts.com/mars/"
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')
table = soup.find("table", id="tablepress-mars")

In [39]:
table_string = str(table)
table_string

'<table class="tablepress tablepress-id-mars" id="tablepress-mars">\n<tbody>\n<tr class="row-1 odd">\n<td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/>\n</td>\n</tr>\n<tr class="row-2 even">\n<td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/>\n</td>\n</tr>\n<tr class="row-3 odd">\n<td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.42 x 10^23 kg (10.7% Earth)</td>\n</tr>\n<tr class="row-4 even">\n<td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/deimos/">Deimos</a>)</td>\n</tr>\n<tr class="row-5 odd">\n<td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km (1.52 AU)</td>\n</tr>\n<tr class="row-6 even">\n<td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/>\n</td>\n</tr>\n<tr clas

In [40]:
data = pd.read_html(table_string)
data = data[0]
data

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [41]:
data = dict(zip(data[0].tolist(), data[1].tolist()))
data

{'Equatorial Diameter:': '6,792 km',
 'First Record:': '2nd millennium BC',
 'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.52 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Polar Diameter:': '6,752 km',
 'Recorded By:': 'Egyptian astronomers',
 'Surface Temperature:': '-153 to 20 °C'}

In [20]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.get(url)

In [21]:
hemisphere_list = ["Cerberus Hemisphere Enhanced", "Schiaparelli Hemisphere Enhanced", "Syrtis Major Hemisphere Enhanced", "Valles Marineris Hemisphere Enhanced"]
img_url_list = []
for hemisphere in hemisphere_list:
    browser.get(url)
    browser.find_element_by_link_text(f"{hemisphere}").click()
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    img_url = soup.find_all("a", text="Sample")[0]['href']
    img_url_list.append(img_url)

In [22]:
hemisphere_image_urls = [
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
]

In [23]:
for dictionary in hemisphere_image_urls:
        dictionary['img_url']

In [24]:
for x in range(len(hemisphere_image_urls)):
    hemisphere_image_urls[x]['img_url'] = img_url_list[x]

In [25]:
def scrape():
    scraped_data = {
        "News Title": news_title,
        "News Paragraph": news_p,
        "Featured Image URL": featured_image_url,
        "Mars Weather": mars_weather,
        "Mars Data": data,
        "Hemisphere Image Urls": hemisphere_image_urls }
    return scraped_data

In [26]:
scraped_data = scrape()

In [27]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [28]:
mars = client.marsDB

In [29]:
results = mars.mission.find()

In [30]:
results_list = []
for row in results:
    results_list.append(row)
results_list

[{'Featured Image URL': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17563_hires.jpg',
  'Hemisphere Image Urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
    'title': 'Cerberus Hemisphere'},
   {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
    'title': 'Schiaparelli Hemisphere'},
   {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
    'title': 'Syrtis Major Hemisphere'},
   {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
    'title': 'Valles Marineris Hemisphere'}],
  'Mars Data': '<table class="tablepress tablepress-id-mars" id="tablepress-mars">\n<tbody>\n<tr class="row-1 odd">\n<td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/>\n</td>\n</tr>\n<tr class="row-2 even">\

In [31]:
try:
    item = mars.mission.find_one()
    item_id = item.get('_id')
    mars.mission.update_one({'_id': item_id}, {"$set": scraped_data}, upsert=True)
except AttributeError: 
    mars.mission.insert_one(scraped_data)

In [32]:
item_id

ObjectId('5ae4f3c2eefbfe6e8f9613ef')

In [60]:
scraped_data

{'Featured Image URL': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17563_hires.jpg',
 'Hemisphere Image Urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
   'title': 'Cerberus Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
   'title': 'Schiaparelli Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
   'title': 'Syrtis Major Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
   'title': 'Valles Marineris Hemisphere'}],
 'Mars Data': {'Equatorial Diameter:': '6,792 km',
  'First Record:': '2nd millennium BC',
  'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
  'Moons:': '2 (Phobos & Deimos)',
  'Orbit Distance:': '227,943,824 km (1.52 AU)',
  'Orbit Period:': '687 days (1.9 years)',
 

In [57]:
results = mars.mission.find()
results

In [58]:
results = mars.mission.find()
results_list = []
for row in results:
    results_list.append(row)
len(results_list)
results_list

[{'Featured Image URL': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA15254_hires.jpg',
  'Hemisphere Image Urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
    'title': 'Cerberus Hemisphere'},
   {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
    'title': 'Schiaparelli Hemisphere'},
   {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
    'title': 'Syrtis Major Hemisphere'},
   {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
    'title': 'Valles Marineris Hemisphere'}],
  'Mars Data': {'Equatorial Diameter:': '6,792 km',
   'First Record:': '2nd millennium BC',
   'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
   'Moons:': '2 (Phobos & Deimos)',
   'Orbit Distance:': '227,943,824 km (1.52 AU)',
   'Orbit Period:': '687 days 

In [59]:
results = mars.mission.find()
results_list = []
for row in results:
    results_list.append(row)
results_list

[{'Featured Image URL': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA15254_hires.jpg',
  'Hemisphere Image Urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
    'title': 'Cerberus Hemisphere'},
   {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
    'title': 'Schiaparelli Hemisphere'},
   {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
    'title': 'Syrtis Major Hemisphere'},
   {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
    'title': 'Valles Marineris Hemisphere'}],
  'Mars Data': {'Equatorial Diameter:': '6,792 km',
   'First Record:': '2nd millennium BC',
   'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
   'Moons:': '2 (Phobos & Deimos)',
   'Orbit Distance:': '227,943,824 km (1.52 AU)',
   'Orbit Period:': '687 days 

In [36]:
data

{'Equatorial Diameter:': '6,792 km',
 'First Record:': '2nd millennium BC',
 'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.52 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Polar Diameter:': '6,752 km',
 'Recorded By:': 'Egyptian astronomers',
 'Surface Temperature:': '-153 to 20 °C'}

In [38]:
data["Equatorial Diameter:"]

'6,792 km'

In [82]:
def viewTable():
    results = mars.mission.find()[0]
    del results["_id"]
    return results

In [84]:
jsonify(viewTable())

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in a way.  To solve
this set up an application context with app.app_context().  See the
documentation for more information.

In [81]:
results = mars.mission.find()
results[0]

{'Featured Image URL': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA15254_hires.jpg',
 'Hemisphere Image Urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
   'title': 'Cerberus Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
   'title': 'Schiaparelli Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
   'title': 'Syrtis Major Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
   'title': 'Valles Marineris Hemisphere'}],
 'Mars Data': {'Equatorial Diameter:': '6,792 km',
  'First Record:': '2nd millennium BC',
  'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
  'Moons:': '2 (Phobos & Deimos)',
  'Orbit Distance:': '227,943,824 km (1.52 AU)',
  'Orbit Period:': '687 days (1.9 years)',
 

In [73]:
len(viewTable()[0])

7